# Full Profile Conjoint Analysis with Max utility rule and logit based market prediction :

In [1]:
#intialise libraries
import pandas as pd 
import numpy as np 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import preprocessing
import matplotlib.pyplot as plt


/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.DataFrame()
#read the input csv 
rank_data = pd.read_csv("/Users/prajwalsreenivas/Downloads/bike_conjoint.csv")
data  = rank_data

In [3]:
rank_data[['gear','type','susp','guards']] = rank_data['Attribute'].str.split(',',expand=True)
rank_data = rank_data.drop(columns=['Attribute'])
rank_data

,Rank,gear,type,susp,guards
0,1,Has gears,Mountain bike,Suspension,Mudguards
1,2,No gears,Mountain bike,Suspension,Mudguards
2,3,Has gears,Racing bike,Suspension,Mudguards
3,4,No gears,Racing bike,Suspension,Mudguards
4,5,Has gears,Mountain bike,No Suspension,Mudguards
5,6,No gears,Mountain bike,No Suspension,Mudguards
6,7,Has gears,Mountain bike,Suspension,No mudguards
7,8,No gears,Mountain bike,Suspension,No mudguards
8,9,Has gears,Racing bike,No Suspension,Mudguards
9,10,No gears,Racing bike,No Suspension,Mudguards


In [4]:
#introduce dummy variables 
conjoint_data = pd.get_dummies(rank_data,columns =['gear','type','susp','guards'])

In [5]:
conjoint_data

,Rank,gear_Has gears,gear_No gears,type_ Mountain bike,type_ Racing bike,susp_ No Suspension,susp_ Suspension,guards_ Mudguards,guards_ No mudguards
0,1,1,0,1,0,0,1,1,0
1,2,0,1,1,0,0,1,1,0
2,3,1,0,0,1,0,1,1,0
3,4,0,1,0,1,0,1,1,0
4,5,1,0,1,0,1,0,1,0
5,6,0,1,1,0,1,0,1,0
6,7,1,0,1,0,0,1,0,1
7,8,0,1,1,0,0,1,0,1
8,9,1,0,0,1,1,0,1,0
9,10,0,1,0,1,1,0,1,0


In [6]:
#renaming columns of dataframe
fullNames = {"Rank":"Rank", \
           "gear_Has gears": "geared","gear_No gears": "fixedgear","type_ Mountain bike": "Mountain bike", \
          "type_ Racing bike": "Road Bike", "susp_ No Suspension":"Hardtail",  "susp_ Suspension":"Softtail", \
           "guards_ Mudguards":"Mudguarded", "guards_ No mudguards":"openmudguard"
          }

conjoint_data.rename(columns=fullNames, inplace=True)

In [7]:
conjoint_data.columns

Index([u'Rank', u'geared', u'fixedgear', u'Mountain bike', u'Road Bike',
       u'Hardtail', u'Softtail', u'Mudguarded', u'openmudguard'],
      dtype='object')

# Estimate maineffects using linear regression 


In [8]:
X = conjoint_data[[ u'geared', u'fixedgear', u'Mountain bike', u'Road Bike',
       u'Hardtail', u'Softtail', u'Mudguarded', u'openmudguard']]
X = sm.add_constant(X)
Y = conjoint_data.Rank
linearRegression = sm.OLS(Y, X).fit()
linearRegression.summary()


/anaconda2/lib/python2.7/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.9380
Date:                Thu, 15 Mar 2018   Prob (F-statistic):              0.477
Time:                        12:13:53   Log-Likelihood:                -36.134
No. Observations:                  16   AIC:                             82.27
Df Residuals:                      11   BIC:                             86.13
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             1.5833      0.233      6.805      0.000       1.071       2.095
geared            0.4167      0.708      0.589      0.568      -1.141       1.974
fixedgear         1.1667      0.708      1.649      0.127      -0.391       2.724
Mountain bike     0.5417      0.708      0.765      0.460      -1.016       2.099
Road Bike         1.0417      0.708      1.472      0.169      -0.516       2.599
Hardtail          2.0417      0.708      2.885      0.015       0.484       3.599
Softtail         -0.4583      0.708     -0.648      0.530      -2.016       1.099
Mudguarded        1.0417      0.708      1.472      0.169      -0.516       2.599
openmudguard      0.5417      0.708      0.765      0.460      -1.016       2.099
==============================================================================
Omnibus:                        3.357   Durbin-Watson:                   0.740
Prob(Omnibus):                  0.187   Jarque-Bera (JB):                2.586
Skew:                           0.933   Prob(JB):                        0.274
Kurtosis:                       2.371   Cond. No.                     6.39e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.17e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [9]:
conjoint_attributes = [u'geared', u'fixedgear', u'Mountain bike', u'Road Bike',
       u'Hardtail', u'Softtail', u'Mudguarded', u'openmudguard']



# Estimate Part Worth and Relative Importances


In [11]:
level_name = []
part_worth = []
part_worth_range = []
end = 1
for item in conjoint_attributes:
    nlevels = len(list(set(conjoint_data[item])))
    level_name.append(list(set(conjoint_data[item])))
    begin = end
    end = begin + nlevels - 1
    new_part_worth = list(linearRegression.params[begin:end])
    new_part_worth.append((-1) * sum(new_part_worth))
    part_worth_range.append(max(new_part_worth) - min(new_part_worth))
    part_worth.append(new_part_worth)
    # end set to begin next iteration



In [12]:
attribute_importance = []
for item in part_worth_range:
    attribute_importance.append(round(100 * (item / sum(part_worth_range)),2))




In [13]:
effect_name_dict = {u'geared':u'geared', u'fixedgear':u'fixedgear', u'Mountain bike':u'Mountain bike', u'Road Bike':u'Road Bike',
       u'Hardtail':u'Hardtail', u'Softtail':u'Softtail', u'Mudguarded':u'Mudguarded', u'openmudguard':u'openmudguard'}




In [14]:
#print out parthworth's for each level
estimates_of_choice = []
index = 0 
for item in conjoint_attributes : 
    print ("\n Attribute : " , effect_name_dict[item])
    print ("\n Importance : " , attribute_importance[index])
    print('    Level Part-Worths')
    for level in range(len(level_name[index])):
        print('       ',level_name[index][level], part_worth[index][level])
    index = index + 1

('\n Attribute : ', u'geared')
('\n Importance : ', 5.75)
    Level Utilities
('       ', 0, 0.41666666666666696)
('       ', 1, -0.41666666666666696)
('\n Attribute : ', u'fixedgear')
('\n Importance : ', 16.09)
    Level Utilities
('       ', 0, 1.166666666666667)
('       ', 1, -1.166666666666667)
('\n Attribute : ', u'Mountain bike')
('\n Importance : ', 7.47)
    Level Utilities
('       ', 0, 0.5416666666666671)
('       ', 1, -0.5416666666666671)
('\n Attribute : ', u'Road Bike')
('\n Importance : ', 14.37)
    Level Utilities
('       ', 0, 1.0416666666666665)
('       ', 1, -1.0416666666666665)
('\n Attribute : ', u'Hardtail')
('\n Importance : ', 28.16)
    Level Utilities
('       ', 0, 2.0416666666666674)
('       ', 1, -2.0416666666666674)
('\n Attribute : ', u'Softtail')
('\n Importance : ', 6.32)
    Level Utilities
('       ', 0, -0.45833333333333376)
('       ', 1, 0.45833333333333376)
('\n Attribute : ', u'Mudguarded')
('\n Importance : ', 14.37)
    Level Utilities
(

In [15]:
#calculating part worths
flattened_list = [val for sublist in utility for val in sublist]
y = pd.Series(flattened_list)
df2=conjoint_data[[ u'geared', u'fixedgear', u'Mountain bike', u'Road Bike',
       u'Hardtail', u'Softtail', u'Mudguarded', u'openmudguard']]
df2 = df2.astype(float)
i=0
for item in conjoint_attributes:
    df2[item]= df2[item]*flattened_list[i]
    i=i+1
df2

,geared,fixedgear,Mountain bike,Road Bike,Hardtail,Softtail,Mudguarded,openmudguard
0,0.416667,-0.000000,1.166667,-0.000000,0.000000,-0.541667,1.041667,-0.000000
1,0.000000,-0.416667,1.166667,-0.000000,0.000000,-0.541667,1.041667,-0.000000
2,0.416667,-0.000000,0.000000,-1.166667,0.000000,-0.541667,1.041667,-0.000000
3,0.000000,-0.416667,0.000000,-1.166667,0.000000,-0.541667,1.041667,-0.000000
4,0.416667,-0.000000,1.166667,-0.000000,0.541667,-0.000000,1.041667,-0.000000
5,0.000000,-0.416667,1.166667,-0.000000,0.541667,-0.000000,1.041667,-0.000000
6,0.416667,-0.000000,1.166667,-0.000000,0.000000,-0.541667,0.000000,-1.041667
7,0.000000,-0.416667,1.166667,-0.000000,0.000000,-0.541667,0.000000,-1.041667
8,0.416667,-0.000000,0.000000,-1.166667,0.541667,-0.000000,1.041667,-0.000000
9,0.000000,-0.416667,0.000000,-1.166667,0.541667,-0.000000,1.041667,-0.000000


# First Choice rule / Maximum Utility Rule 

In [17]:
utility_scores = df2.values.sum(axis=1)
max_utility = np.argmax(utility_scores)




# Utility scores

In [18]:
part_worth

array([ 2.08333333e+00,  1.25000000e+00, -2.50000000e-01, -1.08333333e+00,
        3.16666667e+00,  2.33333333e+00,  4.44089210e-16, -8.33333333e-01,
        8.33333333e-01, -4.44089210e-16, -2.33333333e+00, -3.16666667e+00,
        1.08333333e+00,  2.50000000e-01, -1.25000000e+00, -2.08333333e+00])

In [19]:
print "The index of combination combination with hightest sum of utility scores is " 
print data.ix[max_utility]

The index of combination combination with hightest sum of utility scores is 
Attribute    Has gears, Mountain bike, No Suspension, Mudgu...
Rank                                                         5
gear                                                 Has gears
type                                             Mountain bike
susp                                             No Suspension
guards                                               Mudguards
Name: 4, dtype: object


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [20]:
Bundle = [1,1,0,0,1,0,1,0,1]
print "The best possible Combination of Stimuli would have the highest rank:",\
linearRegression.predict(Bundle)[0]

The best possible Combination of Stimuli would have the highest rank: 3.124999999999999


# Predicting market share by logit model: 


In [22]:
total_utility=0
c= 0.833
for item in utility_scores:
    total_utility = total_utility + np.exp(c*item)

for item in utility_scores:
    probabilty = np.exp(c*item)/total_utility
    itemindex = np.where(utility_scores==item)

    print 'Market share of profile %s is %s ' % (itemindex,probabilty*100)
   

Market share of profile (array([0]),) is 14.310069567146202 
Market share of profile (array([1]),) is 7.147744042054451 
Market share of profile (array([2]),) is 2.048887142280372 
Market share of profile (array([3]),) is 1.0233996973501138 
Market share of profile (array([4]),) is 35.28225412956815 
Market share of profile (array([5]),) is 17.62315134538963 
Market share of profile (array([6]),) is 2.52324756554141 
Market share of profile (array([7]),) is 1.2603382302650652 
Market share of profile (array([8]),) is 5.051642586190243 
Market share of profile (array([9]),) is 2.523247565541408 
Market share of profile (array([10]),) is 0.3612735402556849 
Market share of profile (array([11]),) is 0.1804527072910286 
Market share of profile (array([12]),) is 6.221203986571551 
Market share of profile (array([13]),) is 3.1074323937259645 
Market share of profile (array([14]),) is 0.8907395451699284 
Market share of profile (array([15]),) is 0.44491595565879194 


ps : on how to choose c or confidence factor, A smaller c causes small shares to become larger, and large shares to become smaller having a flattening effect and viceversa with a larger c having a sharpening effect. Usually 
c = 100/[12*max rating on scale] is used